In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Subset, ChainDataset, DataLoader
import torch.nn.functional as F

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True

import pandas as pd
import pickle

import HSIC
import matplotlib.pyplot as plt
import numpy as np
import func
import torchy

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter


# functions to show an image

In [ ]:
a_file = open("PACS.pkl", "rb")
feature_dict = pickle.load(a_file)

In [ ]:
NUM_CLASSES = 7      # 7 classes for each domain: 'dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person'
DATASETS_NAMES = ['photo', 'art_painting', 'cartoon', 'sketch']
CLASSES_NAMES = ['Dog', 'Elephant', 'Giraffe', 'Guitar', 'Horse', 'House', 'Person']
BATCH_SIZE = 64

DOMAIN_DICT = {'photo':[1,0,0,0], 'art_painting':[0,1,0,0], 'cartoon':[0,0,1,0], 'sketch':[0,0,0,1]}

data_sets = {}
data_loaders = {}

training_names = ['photo', 'art_painting', 'cartoon']

data, l, domains = func.dict_to_data(feature_dict, training_names, DOMAIN_DICT) 
train_loader= DataLoader(torchy.Dataset(data, l, domains), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)



valid_names = ['sketch']

data, l, domains = func.dict_to_data(feature_dict, valid_names, DOMAIN_DICT) 
valid_loader = DataLoader(torchy.Dataset(data, l, domains), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)




In [ ]:
net = torchy.Net()
print(net)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_loaders = [data_loaders['photo'], data_loaders['art_painting']]
validate_loader = data_loaders['cartoon']
# print(len(data_loaders['photo']), len(data_loaders['art']))
func.trainHSIC(net, criterion, optimizer, train_loaders, validate_loader, epochs=10)

In [ ]:
dataiter = iter(data_loaders['sketch'])
features, labels = dataiter.next()


print('GroundTruth: ', ' '.join(f'{CLASSES_NAMES[labels[j]]:5s}' for j in range(4)))

outputs = net(features)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{CLASSES_NAMES[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs


# test_loader = cartoon_dataloader
test_loader = data_loaders['sketch']
# test_loader = cartoon_dataloader
PATH = 'saved_model.pth'
test_net = torchy.Net()
test_net.load_state_dict(torch.load(PATH))



with torch.no_grad():
    for data in test_loader:
        features, labels = data
        # calculate outputs by running images through the network
        outputs = test_net(features)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_loader)} test images: {100 * correct // total} %')